# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 16 2023 9:21AM,259429,10,Eme-110569,Emerginnova,Released
1,Mar 16 2023 9:21AM,259429,10,Enova-11515,Emerginnova,Released
2,Mar 16 2023 9:21AM,259429,10,DNMD-36684,Emerginnova,Released
3,Mar 16 2023 9:21AM,259429,10,Enova-11516,Emerginnova,Released
4,Mar 16 2023 9:17AM,259428,10,SOTAH0000939,"Nextsource Biotechnology, LLC",Released
5,Mar 16 2023 9:17AM,259428,10,SONSB0002138,"Nextsource Biotechnology, LLC",Released
6,Mar 16 2023 9:14AM,259427,20,ATR-I000109,"HVL, LLC dba Atrium Innovations",Released
7,Mar 16 2023 9:11AM,259426,20,9235189-BO,"Exact-Rx, Inc.",Released
8,Mar 16 2023 9:11AM,259426,20,9235184-BO,"Exact-Rx, Inc.",Released
9,Mar 16 2023 9:11AM,259426,20,9239439-BO,"Exact-Rx, Inc.",Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
14,259426,Executing,10
15,259426,Released,16
16,259427,Released,1
17,259428,Released,2
18,259429,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
259425,4.0,4.0
259426,10.0,16.0
259427,NaN,1.0
259428,NaN,2.0
259429,NaN,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
259379,0.0,1.0
259413,44.0,2.0
259414,3.0,51.0
259416,0.0,6.0
259419,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
259379,0,1
259413,44,2
259414,3,51
259416,0,6
259419,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,259379,0,1
1,259413,44,2
2,259414,3,51
3,259416,0,6
4,259419,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,259379,,1
1,259413,44,2
2,259414,3,51
3,259416,,6
4,259419,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 16 2023 9:21AM,259429,10,Emerginnova
4,Mar 16 2023 9:17AM,259428,10,"Nextsource Biotechnology, LLC"
6,Mar 16 2023 9:14AM,259427,20,"HVL, LLC dba Atrium Innovations"
7,Mar 16 2023 9:11AM,259426,20,"Exact-Rx, Inc."
33,Mar 16 2023 9:08AM,259425,10,"Digital Brands, LLC"
41,Mar 16 2023 8:58AM,259424,10,"Citieffe, Inc."
42,Mar 16 2023 8:55AM,259423,10,"Methapharm, Inc."
49,Mar 16 2023 8:53AM,259421,10,"Methapharm, Inc."
50,Mar 16 2023 8:51AM,259420,10,Hush Hush
70,Mar 16 2023 8:37AM,259419,16,Sartorius Bioprocess Solutions


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Mar 16 2023 9:21AM,259429,10,Emerginnova,,4
1,Mar 16 2023 9:17AM,259428,10,"Nextsource Biotechnology, LLC",,2
2,Mar 16 2023 9:14AM,259427,20,"HVL, LLC dba Atrium Innovations",,1
3,Mar 16 2023 9:11AM,259426,20,"Exact-Rx, Inc.",10,16
4,Mar 16 2023 9:08AM,259425,10,"Digital Brands, LLC",4,4
5,Mar 16 2023 8:58AM,259424,10,"Citieffe, Inc.",,1
6,Mar 16 2023 8:55AM,259423,10,"Methapharm, Inc.",2,5
7,Mar 16 2023 8:53AM,259421,10,"Methapharm, Inc.",,1
8,Mar 16 2023 8:51AM,259420,10,Hush Hush,,20
9,Mar 16 2023 8:37AM,259419,16,Sartorius Bioprocess Solutions,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 16 2023 9:21AM,259429,10,Emerginnova,4,
1,Mar 16 2023 9:17AM,259428,10,"Nextsource Biotechnology, LLC",2,
2,Mar 16 2023 9:14AM,259427,20,"HVL, LLC dba Atrium Innovations",1,
3,Mar 16 2023 9:11AM,259426,20,"Exact-Rx, Inc.",16,10
4,Mar 16 2023 9:08AM,259425,10,"Digital Brands, LLC",4,4
5,Mar 16 2023 8:58AM,259424,10,"Citieffe, Inc.",1,
6,Mar 16 2023 8:55AM,259423,10,"Methapharm, Inc.",5,2
7,Mar 16 2023 8:53AM,259421,10,"Methapharm, Inc.",1,
8,Mar 16 2023 8:51AM,259420,10,Hush Hush,20,
9,Mar 16 2023 8:37AM,259419,16,Sartorius Bioprocess Solutions,,1


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 16 2023 9:21AM,259429,10,Emerginnova,4,
1,Mar 16 2023 9:17AM,259428,10,"Nextsource Biotechnology, LLC",2,
2,Mar 16 2023 9:14AM,259427,20,"HVL, LLC dba Atrium Innovations",1,
3,Mar 16 2023 9:11AM,259426,20,"Exact-Rx, Inc.",16,10
4,Mar 16 2023 9:08AM,259425,10,"Digital Brands, LLC",4,4


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Mar 16 2023 9:21AM,259429,10,Emerginnova,4.0,NaN
1,Mar 16 2023 9:17AM,259428,10,"Nextsource Biotechnology, LLC",2.0,NaN
2,Mar 16 2023 9:14AM,259427,20,"HVL, LLC dba Atrium Innovations",1.0,NaN
3,Mar 16 2023 9:11AM,259426,20,"Exact-Rx, Inc.",16.0,10.0
4,Mar 16 2023 9:08AM,259425,10,"Digital Brands, LLC",4.0,4.0


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 16 2023 9:21AM,259429,10,Emerginnova,4.0,0.0
1,Mar 16 2023 9:17AM,259428,10,"Nextsource Biotechnology, LLC",2.0,0.0
2,Mar 16 2023 9:14AM,259427,20,"HVL, LLC dba Atrium Innovations",1.0,0.0
3,Mar 16 2023 9:11AM,259426,20,"Exact-Rx, Inc.",16.0,10.0
4,Mar 16 2023 9:08AM,259425,10,"Digital Brands, LLC",4.0,4.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1815970,37.0,6.0
16,259419,0.0,1.0
18,259379,1.0,0.0
19,778243,59.0,47.0
20,518853,17.0,10.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1815970,37.0,6.0
1,16,259419,0.0,1.0
2,18,259379,1.0,0.0
3,19,778243,59.0,47.0
4,20,518853,17.0,10.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,37.0,6.0
1,16,0.0,1.0
2,18,1.0,0.0
3,19,59.0,47.0
4,20,17.0,10.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,37.0
1,16,Released,0.0
2,18,Released,1.0
3,19,Released,59.0
4,20,Released,17.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,18,19,20
Status,,,,,
Executing,6.0,1.0,0.0,47.0,10.0
Released,37.0,0.0,1.0,59.0,17.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,18,19,20
0,Executing,6.0,1.0,0.0,47.0,10.0
1,Released,37.0,0.0,1.0,59.0,17.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,18,19,20
0,Executing,6.0,1.0,0.0,47.0,10.0
1,Released,37.0,0.0,1.0,59.0,17.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()